# Neural Network

## Imports

In [8]:
import pandas as pd
from sqlalchemy import create_engine

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()


Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\engine\default.py", line 667, in do_rollback
    dbapi_connection.rollback()
  File "src\pymssql\_pymssql.pyx", line 316, in pymssql._pymssql.Connection.rollback
  File "src\pymssql\_pymssql.pyx", line 300, in pymssql._pymssql.Connection.rollback
  File "src\pymssql\_mssql.pyx", line 1056, in pymssql._mssql.MSSQLConnection.execute_non_query
  File "src\pymssql\_mssql.pyx", line 1077, in pymssql._mssql.MSSQLConnection.execute_non_query
  File "src\pymssql\_mssql.pyx", line 1251, in pymssql._mssql.MSSQLConnection.format_and_run_query
  File "s

## EXAMPLE: TRAIN FROM 2021-12, PREDICT FOR 2022-01

In [9]:
df = pd.read_sql(
    """
    select ticker, date, agr, bm, idiovol, mom12m, roeq, ret
    from data
    where date in ('2021-12', '2022-01')
    """, 
    conn
)
features = ["agr", "bm", "idiovol", "mom12m", "roeq"]

## TRANSFORM EACH CROSS-SECTION

In [10]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution="normal")

def qtxs(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

df[features + ["ret"]] = df.groupby(
  "date", 
  group_keys=False
)[features + ["ret"]].apply(qtxs)

## FITTING A NEURAL NETWORK

In [11]:
from sklearn.neural_network import MLPRegressor

df = df.dropna()

Xtrain = df[df.date=='2021-12'][features]
ytrain = df[df.date=='2021-12']["ret"]

model = MLPRegressor(
  hidden_layer_sizes=(4, 2),
  random_state=0
)
model.fit(Xtrain, ytrain)

c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPRegressor(hidden_layer_sizes=(4, 2), random_state=0)

### R-squared on training data

In [12]:
model.score(Xtrain, ytrain)

0.1786475140715703

### R-squared on test data

In [13]:
Xtest = df[df.date=='2022-01'][features]
ytest = df[df.date=='2022-01']["ret"]
model.score(Xtest, ytest)

0.06968891280266065

In [14]:
train_score = []
test_score = []
labels = []
for i in range(2, 16, 2):
    for j in range(2, 16, 2):
        
        model = MLPRegressor(
            hidden_layer_sizes=(i, j),
            random_state=0
            )
        model.fit(Xtrain, ytrain)
        train_score.append(model.score(Xtrain, ytrain))
        test_score.append(model.score(Xtest, ytest))
        labels.append(f'({i}, {j})')
        for k in range(2, 16, 2):
            model = MLPRegressor(
            hidden_layer_sizes=(i, j, k),
            random_state=0
            )
            model.fit(Xtrain, ytrain)
            train_score.append(model.score(Xtrain, ytrain))
            test_score.append(model.score(Xtest, ytest))
            labels.append(f'({i}, {j}, {k})')

c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged

In [22]:
import plotly.graph_objects as go

# Create a trace for the scatter plot
trace = go.Scatter(
    x=train_score,
    y=test_score,
    mode='markers',
    marker=dict(
        size=5,
        sizemode='diameter',
        sizeref=0.1,
        sizemin=5,
        color=train_score,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Training Score')
    ),
    text=['NN size: {}'.format(size) for size in labels]
)

# Create a layout for the plot
layout = go.Layout(
    title='Test Score vs. Training Score for Neural Network',
    xaxis=dict(title='Training Score'),
    yaxis=dict(title='Test Score')
)

# Combine the trace and layout into a figure and display it
fig = go.Figure(data=[trace], layout=layout)
fig.show()